In [1]:
import numpy as np
import random

from training_utils import *

In [2]:
import numpy as np

def riempi_sezioni_vuote(vettore, dim_a, dim_b):
    """
    Sostituisce le sezioni nulle con la sezione successiva.
    Assume che il vettore sia composto da 4 blocchi A e 4 blocchi B.
    """
    # Creiamo una copia per non modificare l'originale direttamente
    risultato = vettore.copy()
    
    # Calcoliamo i punti di inizio per le sezioni A e B
    # Sezioni A: da 0 a 4*dim_a
    # Sezioni B: da 4*dim_a in poi
    
    def processa_gruppo(dati, size, offset):
        # Iteriamo a ritroso dalla sezione 3 alla 1 (indice 2 a 0)
        # Usiamo la sezione successiva (i+1) per coprire quella corrente (i)
        for i in range(2, -1, -1):
            inizio_corr = offset + (i * size)
            fine_corr = inizio_corr + size
            
            inizio_succ = offset + ((i + 1) * size)
            fine_succ = inizio_succ + size
            
            # Estraiamo la sezione corrente
            sezione_attuale = dati[inizio_corr:fine_corr]
            
            # Controlliamo se tutti gli elementi sono zero
            if np.all(sezione_attuale == 0):
                dati[inizio_corr:fine_corr] = dati[inizio_succ:fine_succ]
                
    # Applichiamo la logica alle prime 4 sezioni (A)
    processa_gruppo(risultato, dim_a, 0)
    
    # Applichiamo la logica alle successive 4 sezioni (B)
    offset_b = 4 * dim_a
    processa_gruppo(risultato, dim_b, offset_b)
    
    return risultato

# --- ESEMPIO DI UTILIZZO ---
# Creiamo un vettore di esempio (dim_a=2, dim_b=1)
# Sezioni: [a1, a2, a3, a4, b1, b2, b3, b4]
esempio = np.array([1,1, 0,0, 0,0, 5,5, 0, 0, 9, 9]) 
# Qui: a2 e a3 sono zero, b1 e b2 sono zero.
# Risultato atteso: a3 prende a4, a2 prende il nuovo a3, etc.

print("Vettore originale:", esempio)
nuovo_vettore = riempi_sezioni_vuote(esempio, 2, 1)
print("Vettore modificato:", nuovo_vettore)

Vettore originale: [1 1 0 0 0 0 5 5 0 0 9 9]
Vettore modificato: [1 1 5 5 5 5 5 5 9 9 9 9]


In [12]:
import argparse
import sys

def parse_args(default_config_path="./train_config/train_general.yaml"):
    """
    Parse arguments from CLI or notebook and return a dictionary.
    """
    # --- Gestione notebook ---
    argv = sys.argv[1:]
    if len(argv) == 0 or "--f=" in " ".join(argv):
        argv = [default_config_path]

    # --- Pre-parser per leggere il config_path ---
    pre_parser = argparse.ArgumentParser(add_help=False)
    pre_parser.add_argument(
        "config_path",
        type=str,
        nargs="?",
        default=default_config_path,
        help="Main config file path"
    )
    initial_args, remaining_argv = pre_parser.parse_known_args(argv)
    CONFIG_PATH = initial_args.config_path
    
    # --- Legge parametri dal file di config ---
    # Nota: Assicurati che parse_config_file e str2bool siano definiti nel tuo script
    file_config_dict = parse_config_file(CONFIG_PATH)

    # --- Parser principale ---
    parser = argparse.ArgumentParser(description="Training Script")
    parser.add_argument(
        "config_path",
        type=str,
        nargs="?",
        default=CONFIG_PATH,
        help="Main config file path"
    )

    # Aggiunge parametri dal config file dinamici
    for key, value in file_config_dict.items():
        if isinstance(value, bool):
            parser.add_argument(f"--{key}", type=str2bool, default=value)
        elif value is None:
            parser.add_argument(f"--{key}", type=str, default=value)
        else:
            parser.add_argument(f"--{key}", type=type(value), default=value)

    # --- Parse finale ---
    args_namespace, unknown = parser.parse_known_args(remaining_argv)
    
    if unknown:
        print("Ignored unknown args:", unknown)

    # --- Conversione da Namespace a Dizionario ---
    # vars() trasforma l'oggetto Namespace in un dict { 'parametro': valore }
    args_dict = vars(args_namespace)
    
    return args_dict

In [13]:
a = parse_args()

In [14]:
a

{'config_path': './train_config/train_general.yaml',
 'training': {'exp_name': 'old_pers',
  'machine_name': 'personal',
  'seed': 180618,
  'cuda': 0,
  'torch_deterministic': False,
  'wandb': True,
  'worker_id': 0,
  'env_id': '3xold',
  'n_envs': 3,
  'build_path': './unity_build/3xold_wind/UASRL.exe',
  'headless': False,
  'test_lib': False,
  'base_time': 1765457030,
  'total_timesteps': 50000,
  'learning_starts': 1000,
  'buffer_size': 120000,
  'batch_size': 256,
  'bootstrap': True,
  'bootstrap_batch_proportion': 0.8,
  'input_stack': 4,
  'q_ensemble_n': 5,
  'actor_network_layers': [128, 128, 128],
  'q_network_layers': [128, 128, 128],
  'gamma': 0.995,
  'tau': 0.005,
  'reward_scale': 1.0,
  'policy_lr': 0.0004,
  'q_lr': 0.0004,
  'alpha_lr': 0.0004,
  'noise_clip': 0.5,
  'alpha': 0.2,
  'target_entropy': -2.0,
  'autotune': True,
  'update_frequency': 1,
  'policy_frequency': 4,
  'target_network_update_period': 1,
  'loss_log_interval': 500,
  'metrics_log_interva